In [1]:
import pandas as pd
import numpy as np

import torchaudio
import torch

from torch.fft import rfft as fft
from torch.fft import irfft as ifft

from tqdm.auto import tqdm
import soundfile as sf

import IPython

import collections
import os

from filters.frequency_domain import single_FDAF

In [2]:
path = './data/midi_noise_cut'

In [3]:
lines = np.loadtxt(os.path.join(path, "dir_list.txt"), comments="#", unpack=False, dtype=str)

In [4]:
dirs = collections.defaultdict(list)
for line in lines:
    l = []
    for (dirpath, dirnames, filenames) in os.walk(os.path.join(path, line)):
        l.append(filenames)
    dirs[line] = l

In [22]:
dirs[list(dirs.keys())[15]][0]

['fdaf.wav',
 'meta.json',
 'raw_mic_0.wav',
 'raw_mic_1.wav',
 'raw_mic_2.wav',
 'raw_mic_3.wav',
 'raw_spk_0.wav',
 'raw_spk_1.wav',
 'vqe_0.wav']

In [12]:
def get_names(ind, path):
    input_name = dirs[list(dirs.keys())[ind]][0][-7]
    response_name = dirs[list(dirs.keys())[ind]][0][-2]
    dir_name = list(dirs.keys())[ind]
    return os.path.join(os.path.join(path, dir_name), input_name),\
os.path.join(os.path.join(path, dir_name), response_name), os.path.join(os.path.join(path, dir_name))

In [13]:
for i in tqdm(range(len(dirs))):
    input_seq, response_seq, save_path = get_names(i, path)
    x, x_sr = torchaudio.load(input_seq)
    d, d_sr = torchaudio.load(response_seq)
    x, d = x.squeeze(), d.squeeze()

    y, e = single_FDAF(x, d, 256, 0.1)
    audio_path = os.path.join(save_path,'fdaf.wav')
    sf.write(audio_path, y, x_sr, subtype='PCM_16')
    #IPython.display.Audio(audio_path, rate=x_sr)

  0%|          | 0/847 [00:00<?, ?it/s]

  0%|          | 0/802 [00:00<?, ?it/s]

  0%|          | 0/814 [00:00<?, ?it/s]

  0%|          | 0/811 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?it/s]

...

  0%|          | 0/937 [00:00<?, ?it/s]

  0%|          | 0/810 [00:00<?, ?it/s]

  0%|          | 0/801 [00:00<?, ?it/s]

  0%|          | 0/811 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [128]:
from scipy.linalg import hankel

In [ ]:
def mono_Kalman(x, d, N, delta=1e-4):
    

In [226]:
def mono_Kalman(x, d, N, delta=1e-4):
    n_blocks = x.shape[-1]-N
    
    u = torch.zeros(N)
    w = torch.zeros(N)
    Q = torch.eye(N)*delta
    P = torch.eye(N)*delta
    I = torch.eye(N)
    
    e = torch.zeros(n_blocks)
    y = torch.zeros(n_blocks)
    for k in tqdm(range(n_blocks)):
        u[1:] = u.clone()[:-1]
        u[0] = x[k]
        y[k] = u @ w
        e_t = d[k] - y[k]
        R = e_t**2+1e-10
        P_t = P + Q
        r = P_t @ u
        K = r/((u@r)+R+1e-10)
        w = w + (K*e_t)
        P = (I-torch.outer(K, u)) @ P_t
        e[k]=e_t
    return e, y

In [227]:
input_seq, response_seq, save_path = get_names(0, path)
x, x_sr = torchaudio.load(input_seq)
d, d_sr = torchaudio.load(response_seq)
x, d = x.squeeze(), d.squeeze()

e,y = mono_Kalman(x, d, 128, 1e-8)


  0%|          | 0/205184 [00:00<?, ?it/s]

In [228]:
audio_path = os.path.join(save_path,'nlms.wav')
sf.write(audio_path, y, x_sr, subtype='PCM_16')
IPython.display.Audio(audio_path, rate=x_sr)